In [64]:
import pandas  as pd 
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
from datetime import datetime as dt 
import warnings
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, f1_score, precision_score, roc_auc_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

In [38]:
drugs = pd.read_csv('drugs_train.csv')
drugs.info()
drugs.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1500 non-null   object 
 1   age                             1500 non-null   object 
 2   gender                          1500 non-null   object 
 3   education                       1500 non-null   object 
 4   country                         1500 non-null   object 
 5   ethnicity                       1500 non-null   object 
 6   personality_neuroticism         1500 non-null   float64
 7   personality_extraversion        1500 non-null   float64
 8   personality_openness            1500 non-null   float64
 9   personality_agreeableness       1500 non-null   float64
 10  personality_conscientiousness   1500 non-null   float64
 11  personality_impulsiveness       1500 non-null   float64
 12  personality_sensation           15

Index(['id', 'age', 'gender', 'education', 'country', 'ethnicity',
       'personality_neuroticism', 'personality_extraversion',
       'personality_openness', 'personality_agreeableness',
       'personality_conscientiousness', 'personality_impulsiveness',
       'personality_sensation', 'consumption_alcohol',
       'consumption_amphetamines', 'consumption_caffeine',
       'consumption_cannabis', 'consumption_chocolate',
       'consumption_mushrooms', 'consumption_nicotine',
       'consumption_cocaine_last_month'],
      dtype='object')

In [34]:
drugs.education.value_counts()

Some college or university, no certificate or degree    405
University degree                                       376
Masters degree                                          229
Professional certificate/ diploma                       221
Left school at 18 years                                  85
Left school at 16 years                                  72
Doctorate degree                                         66
Left school at 17 years                                  26
Left school before 16 years                              20
Name: education, dtype: int64

In [67]:
#print(drugs.describe())
def getting_dummies(drugs):
    drugs['tmp'] = range(0,drugs.shape[0])
    t1 = pd.get_dummies(drugs['age'])
    t1['tmp'] = range(0,drugs.shape[0])
    drugs = pd.merge(drugs, t1, on = ['tmp'])

    t1 = pd.get_dummies(drugs['country'])
    t1['tmp'] = range(0,drugs.shape[0])
    drugs = pd.merge(drugs, t1, on = ['tmp'])
    drugs['Other_country'] = drugs['Other'] + drugs['Ireland'] + drugs['Canada']

    t1 = pd.get_dummies(drugs['ethnicity'])
    t1['tmp'] = range(0,drugs.shape[0])
    drugs = pd.merge(drugs, t1, on = ['tmp'])

    drugs = drugs.drop(columns = ['tmp', 'age', 'country', 'ethnicity','Ireland', 'Canada', 'Other_x' ])

    return (drugs)

def getting_ordinals(drugs):
    
    lst = ['consumption_alcohol', 'consumption_amphetamines', 'consumption_caffeine',
            'consumption_cannabis', 'consumption_chocolate',
            'consumption_mushrooms', 'consumption_nicotine']
    consDict = {'never used':0,  'used over a decade ago':1,
       'used in last decade':2,'used in last year':3, 'used in last month':4, 'used in last week' : 5, 'used in last day' : 6}

    eduDict = {'Left school before 16 years' : 0, 'Left school at 16 years' : 0, 'Left school at 17 years':0, 'Left school at 18 years' : 0, 
               'Some college or university, no certificate or degree':1, 'Professional certificate/ diploma':2 ,'University degree':3, 'Masters degree':4, 
               'Doctorate degree' : 5}


    for cols in lst:
        drugs[cols]=drugs[cols].map(consDict)
    
    drugs['education'] = drugs['education'].map(eduDict)
    return(drugs)

def gender_dummy(drugs):
    drugs['female'] = np.where(drugs['gender'] == 'female', 1, 0) 
    drugs = drugs.drop(columns=['gender', 'id'])
    return (drugs)

def expl_var_dummy(drugs):
    drugs['consumption_cocaine_last_month'] = np.where(drugs['consumption_cocaine_last_month'] == 'Yes', 1, 0) 
    return (drugs)

def scalling(drugs):
    scaled_features = drugs.copy()

    col_names = ['personality_neuroticism', 'personality_extraversion',
                'personality_openness', 'personality_agreeableness',
                'personality_conscientiousness', 'personality_impulsiveness',
                'personality_sensation']
    features = scaled_features[col_names]

    scaler = RobustScaler().fit(features.values)
    features = scaler.transform(features.values)

    scaled_features[col_names] = features

    return(scaled_features)
    
def our_metrics(y_test, preds):
     print(f'Balanced Accuracy:', balanced_accuracy_score(y_test, preds), 
          '\nconfusion:', confusion_matrix(y_test, preds),
          '\nprecision:', precision_score(y_test, preds) ,
          '\naccuracy:', accuracy_score(y_test, preds), 
          '\nrecall:', recall_score(y_test, preds),
          '\nauroc:', roc_auc_score(y_test, preds) )
    

In [73]:
drugs = pd.read_csv('drugs_train.csv')

drugs = getting_dummies(drugs)
drugs = getting_ordinals(drugs)
drugs = gender_dummy(drugs)
drugs = expl_var_dummy(drugs)
drugs = scalling(drugs)
x_train, x_test, y_train, y_test = train_test_split(drugs.loc[:,drugs.columns!='consumption_cocaine_last_month'], drugs.consumption_cocaine_last_month, test_size=0.25)

print(drugs.columns )
drugs.head()


Index(['education', 'personality_neuroticism', 'personality_extraversion',
       'personality_openness', 'personality_agreeableness',
       'personality_conscientiousness', 'personality_impulsiveness',
       'personality_sensation', 'consumption_alcohol',
       'consumption_amphetamines', 'consumption_caffeine',
       'consumption_cannabis', 'consumption_chocolate',
       'consumption_mushrooms', 'consumption_nicotine',
       'consumption_cocaine_last_month', '18-24', '25-34', '35-44', '45-54',
       '55-64', '65+', 'Australia', 'New Zealand', 'UK', 'USA',
       'Other_country', 'Asian', 'Black', 'Mixed-Black/Asian',
       'Mixed-White/Asian', 'Mixed-White/Black', 'Other_y', 'White', 'female'],
      dtype='object')


,education,personality_neuroticism,personality_extraversion,personality_openness,personality_agreeableness,personality_conscientiousness,personality_impulsiveness,personality_sensation,consumption_alcohol,consumption_amphetamines,consumption_caffeine,consumption_cannabis,consumption_chocolate,consumption_mushrooms,consumption_nicotine,consumption_cocaine_last_month,18-24,25-34,35-44,45-54,55-64,65+,Australia,New Zealand,UK,USA,Other_country,Asian,Black,Mixed-Black/Asian,Mixed-White/Asian,Mixed-White/Black,Other_y,White,female
0,4,0.287179,0.354680,-0.111588,-0.115607,0.213483,0.000000,-0.978328,5,1,6,5,6,0,5,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
1,3,-0.215385,0.832512,-0.300429,-0.115607,0.342697,-0.396476,-0.718266,5,0,5,0,6,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,3,0.287179,-0.334975,0.111588,-0.242775,0.000000,0.889868,0.247678,4,0,6,5,5,3,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
3,4,1.015385,-0.931034,-0.390558,0.375723,-1.016854,0.889868,0.529412,6,0,6,2,6,0,2,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
4,1,0.210256,0.600985,0.751073,0.942197,-0.421348,0.334802,0.247678,5,0,4,4,6,3,4,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [59]:
y_test.value_counts()

0    137
1     13
Name: consumption_cocaine_last_month, dtype: int64

In [74]:

reg = LogisticRegression()
reg.fit(x_train, y_train)

reg_pred = reg.predict(x_test)

our_metrics(y_test, reg_pred)


Balanced Accuracy: 0.5528571428571429 
confusion: [[345   5]
 [ 22   3]] 
precision: 0.375 
accuracy: 0.928 
recall: 0.12 
auroc: 0.5528571428571429


In [77]:
import lightgbm as lgb
lg = lgb.LGBMClassifier()
lg.fit(x_train, y_train)
lgb_preds = lg.predict(x_test)

our_metrics(y_test, lgb_preds)



Balanced Accuracy: 0.51 
confusion: [[343   7]
 [ 24   1]] 
precision: 0.125 
accuracy: 0.9173333333333333 
recall: 0.04 
auroc: 0.51


In [184]:
import numpy as np
import optuna

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    
    #train_x, valid_x, train_y, valid_y = train_test_split(drugs.loc[:,drugs.columns!='consumption_cocaine_last_month'], drugs.consumption_cocaine_last_month, test_size=0.1, random_state = 112)
    #dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "n_estimators": trial.suggest_int("n_estimators", 1000, 8000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.5, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.5, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 4000, 12000),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 140),
    }

    lst = []
    for i in range(10):
        train_x, valid_x, train_y, valid_y = train_test_split(drugs.loc[:,drugs.columns!='consumption_cocaine_last_month'], drugs.consumption_cocaine_last_month, test_size=0.1, random_state = i+1)
        dtrain = lgb.Dataset(train_x, label=train_y)

        gbm = lgb.train(param, dtrain)
        preds = gbm.predict(valid_x)
        #pred_labels = np.rint(preds)
        pred_labels1 = np.where(preds>0.4,1,0)
        # pred_labels2 = np.where(preds>0.4,1,0)
        # pred_labels3 = np.where(preds>0.2,1,0)
        # pred_labels4 = np.where(preds>0.1,1,0)
        x1 = balanced_accuracy_score(valid_y, pred_labels1)
        # x2 = balanced_accuracy_score(valid_y, pred_labels2)
        # x3 = balanced_accuracy_score(valid_y, pred_labels3)
        # x4 = balanced_accuracy_score(valid_y, pred_labels4)
        pred_labels = x1 #max(x1, x2, x3, x4)
        lst.append(pred_labels)
        #lst.append(balanced_accuracy_score(valid_y, pred_labels))
    accuracy = np.mean(lst) 
        #sklearn.metrics.balanced_accuracy_score(valid_y, pred_labels)
        
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-05-22 09:36:57,975] A new study created in memory with name: no-name-3f14d5c3-e50d-4a31-91a7-ac2619a8b579
[I 2022-05-22 09:38:34,548] Trial 0 finished with value: 0.5666087553417968 and parameters: {'n_estimators': 2941, 'learning_rate': 0.1547358871633316, 'lambda_l1': 5.304610961295152e-06, 'lambda_l2': 1.1194096153140269e-07, 'num_leaves': 10680, 'feature_fraction': 0.8217196196456307, 'bagging_fraction': 0.4313792263467209, 'bagging_freq': 4, 'min_child_samples': 11}. Best is trial 0 with value: 0.5666087553417968.
[I 2022-05-22 09:39:50,298] Trial 1 finished with value: 0.5808095865933541 and parameters: {'n_estimators': 2369, 'learning_rate': 0.13039588734289487, 'lambda_l1': 2.0067864345751194e-08, 'lambda_l2': 1.0237127431897417, 'num_leaves': 8558, 'feature_fraction': 0.40875941861490606, 'bagging_fraction': 0.591520379532485, 'bagging_freq': 2, 'min_child_samples': 12}. Best is trial 1 with value: 0.5808095865933541.
[I 2022-05-22 09:40:46,290] Trial 2 finished with v

Number of finished trials: 100
Best trial:
  Value: 0.6400432903523962
  Params: 
    n_estimators: 6756
    learning_rate: 0.23307573473652426
    lambda_l1: 0.2088746549420361
    lambda_l2: 0.0017888964399385525
    num_leaves: 7083
    feature_fraction: 0.5342126511516654
    bagging_fraction: 0.43315871661958205
    bagging_freq: 3
    min_child_samples: 137


In [175]:
study.best_trial.params


{'n_estimators': 6584,
 'learning_rate': 0.2585180342596222,
 'lambda_l1': 0.24869615891449395,
 'lambda_l2': 1.5185272830960025e-07,
 'num_leaves': 5328,
 'feature_fraction': 0.5516054799322556,
 'bagging_fraction': 0.9202901878168293,
 'bagging_freq': 7,
 'min_child_samples': 127}

In [190]:
param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "n_estimators":study.best_trial.params['n_estimators'],
        "learning_rate": study.best_trial.params['learning_rate'],
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": study.best_trial.params['lambda_l1'],
        "lambda_l2": study.best_trial.params['lambda_l2'] ,
        "num_leaves": study.best_trial.params['num_leaves'],
        "feature_fraction": study.best_trial.params['feature_fraction'],
        "bagging_fraction": study.best_trial.params['bagging_fraction'],
        "bagging_freq": study.best_trial.params['bagging_freq'],
        "min_child_samples": study.best_trial.params['min_child_samples'],
    }

lst = []
for i in range(200):
    train_x, valid_x, train_y, valid_y = train_test_split(drugs.loc[:,drugs.columns!='consumption_cocaine_last_month'], drugs.consumption_cocaine_last_month, test_size=0.15, random_state = i+1*11)
    dtrain = lgb.Dataset(train_x, label=train_y)

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.where(preds>0.4,1,0)
    lst.append(balanced_accuracy_score(valid_y, pred_labels))

    our_metrics(valid_y, pred_labels)
print(np.mean(lst))

Balanced Accuracy: 0.5162505380972879 
confusion: [[191  11]
 [ 21   2]] 
precision: 0.15384615384615385 
accuracy: 0.8577777777777778 
recall: 0.08695652173913043 
auroc: 0.5162505380972879
Balanced Accuracy: 0.5597287989668531 
confusion: [[191  11]
 [ 19   4]] 
precision: 0.26666666666666666 
accuracy: 0.8666666666666667 
recall: 0.17391304347826086 
auroc: 0.5597287989668531
Balanced Accuracy: 0.5210781808891161 
confusion: [[193  13]
 [ 17   2]] 
precision: 0.13333333333333333 
accuracy: 0.8666666666666667 
recall: 0.10526315789473684 
auroc: 0.521078180889116
Balanced Accuracy: 0.6159825870646767 
confusion: [[189  12]
 [ 17   7]] 
precision: 0.3684210526315789 
accuracy: 0.8711111111111111 
recall: 0.2916666666666667 
auroc: 0.6159825870646767
Balanced Accuracy: 0.5809913132345427 
confusion: [[196  10]
 [ 15   4]] 
precision: 0.2857142857142857 
accuracy: 0.8888888888888888 
recall: 0.21052631578947367 
auroc: 0.5809913132345427
Balanced Accuracy: 0.5486211901306242 
confusion:

In [141]:
from sklearn.neighbors import KNeighborsClassifier
train_x, valid_x, train_y, valid_y = train_test_split(drugs.loc[:,drugs.columns!='consumption_cocaine_last_month'], drugs.consumption_cocaine_last_month, test_size=0.1, random_state = 997)
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
knn_pred = knn.predict(valid_x)

our_metrics(valid_y, knn_pred)

Balanced Accuracy: 0.572463768115942 
confusion: [[135   3]
 [ 10   2]] 
precision: 0.4 
accuracy: 0.9133333333333333 
recall: 0.16666666666666666 
auroc: 0.5724637681159421


In [142]:
!py -m pip install catboost

     ---------------------------------------- 73.9/73.9 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 15.2/15.2 MB 1.8 MB/s eta 0:00:00
